In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
# utility classes

import json

class LogFile(object):
    
    def __init__(self, json_content):
        self._steps = [Step(s) for s in json_content]
        if len(self._steps) <= 0:
            self._max_length = 0
        else:
            self._max_length = max([s._max_length for s in self._steps])
    
    def __repr__(self):
        s = "LogFile{"
        for step in self._steps:
            s += "%s," % step
        if len(self._steps) > 0:
            s = s[:-1]
        s += "}"
        return s
    
    @staticmethod
    def parseJsonFile(filename):
        with open(filename, 'r') as f:
            json_content = json.load(f)
        return LogFile(json_content)

class Step(object):
    def __init__(self, json_content):
        self._sender_count = json_content["sendercount"]
        self._cliques = json_content["clqs"]
        self._last_messages = [View(j) for j in json_content["lms"]]
        self._max_length = max([v.get_max_length() for v in self._last_messages])
    
    def __repr__(self):
        s = "Step{sender_count: %d, cliques: %s, last_messages: %s}" % (
            self._sender_count, self._cliques, self._last_messages)
        return s
    
    def show_plot(self, max_length):
        
        f, axes = plt.subplots(self._sender_count, sharex=True, sharey=True)
        for i, view in enumerate(self._last_messages):
            view.add_plot(self._sender_count, max_length, axes[i])
        plt.setp(axes, xticks=range(1, max_length), yticks=range(0, self._sender_count))
        #plt.xticks(range(1, max_length))
        #plt.yticks(range(0, self._sender_count))
        plt.show()

class View(object):
    def __init__(self, json_content):
        _json_content = json_content[0]
        #print(_json_content)
        self._heights = {}
        self._messages = {key: Message(value) for (key, value) in _json_content.items()}
        self.compute_heights()
    
    def __repr__(self):
        return "View%s" % self._messages
    
    def add_plot(self, sender_count, max_length, axes):
        #print(self)
        #print(self._messages.keys())
        #f, array_axes = plt.subplot(sender_count, sharex=True, sharey=True)
        #axes.set_xticks(range(1, max_length))
        #axes.set_yticks(range(0, sender_count))
        
        for i, key in enumerate(sorted(self._messages.keys())):
            self._messages[key].add_plot(sender_count, self._heights, axes)
        #for key, message in self._messages.items():
        #    message.add_plot(sender_count, self._heights)
            
        #plt.show()
            
    def get_max_length(self):
        return max([len(m._justification) for m in self._messages.values()])
    
    def compute_heights(self):
        for name, message in self._messages.items():
            for index, block in enumerate(reversed(message._justification)):
                if block is not None:
                    self._heights[block[1]] = index
                    last_block = block
                else:
                    self._heights[None] = -1
 

class Message(object):
    def __init__(self, json_content):
        # formatting?
        assert len(json_content) == 1
        # get first key
        self._name, justification = json_content.popitem()
        self._justification = [tuple(j) if j != 'None' else None for j in justification]
    
    def __repr__(self):
        return "Message{name: %s, justification: %s}" % (self._name, self._justification)
    
    def add_plot(self, sender_count, heights, axes):
        #print(self)
        # m is None if its the genesis remove "if m is not None" to show it on the graphs
        x = [heights[m[1]] if m is not None else heights[m] for m in self._justification if m is not None]
        y = [-1 if m is None else int(m[0]) for m in self._justification if m is not None]
        axes.plot(x,y, 'bo', linestyle='solid')
    
    

In [ ]:
# json files loading

from os import listdir
from os.path import isfile, join

directory = "./generated"

# all files (no folders) in directory
onlyfiles = sorted([f for f in listdir(directory) if isfile(join(directory, f))])

log_files = [LogFile.parseJsonFile(directory+'/'+f) for f in onlyfiles]

# filter out empty files
log_files = [log_file for log_file in log_files if len(log_file._steps) > 0]


    

In [ ]:
for log_file in log_files:
    
    for step in log_file._steps:
        #print(step)
        step.show_plot(log_file._max_length)
        print("-----")
    break

In [ ]:

x='''m1 = [(1,0), (2,1), (1,2)]
m2 = [(2,0), (3,1), (1,2)]
m3 = [(3,0), (3,1), (1,2)]

messages = [m1, m2, m3]

for sender in messages:
    plt.plot(
        [m[0] for m in sender],
        [m[1] for m in sender],
        'bx',
        linestyle='solid'
    )

plt.xticks(range(1,1 + len(messages[0])))
plt.yticks(range(0, len(messages[0])))
plt.show()'''